In [108]:
from fyers_apiv3 import fyersModel
from datetime import datetime , timedelta
import pandas as pd
import pytz
import numpy as np

In [96]:
client_id = "MACO3YJA7I-100"  # Replace with your client ID
secret_key = "N7SXUFQG91"  # Replace with your secret key
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"  # Replace with your redirect URI
response_type = "code" 
grant_type = "authorization_code"  
state = "sample_state"

In [102]:
# # Generate Auth Code
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)
genrate_authcode = session.generate_authcode()
print(genrate_authcode)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=MACO3YJA7I-100&redirect_uri=https%3A%2F%2Ftrade.fyers.in%2Fapi-login%2Fredirect-uri%2Findex.html&response_type=code&state=None


In [103]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDM4NDY3MjgsImV4cCI6MTcwMzg3NjcyOCwibmJmIjoxNzAzODQ2MTI4LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJZSjAwMTI5Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiMjdlMDQ2OGE4MGJkYmVmMGQyYTg3M2MxMmNlN2YyZGEyMjA1NzQ0ZTIwYTg1MDI0YjhlMDA4NDciLCJub25jZSI6IiIsImFwcF9pZCI6Ik1BQ08zWUpBN0kiLCJ1dWlkIjoiMjc1OTcwZWUzNTI2NGMzZGJhNjQ5ODY2ZTZlY2VkNDQiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.RnfHnP86Uwr280zmsQ9RlP-DKrmcxymrm8iD5VaKz6E"

In [105]:
# session = fyersModel.SessionModel(
#     client_id=client_id,
#     secret_key=secret_key, 
#     redirect_uri=redirect_uri, 
#     response_type=response_type, 
#     grant_type=grant_type
# )
# session.set_token(auth_code)
# response = session.generate_token()
# access_token = response['access_token']

In [106]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

## ALL CODE

In [229]:
from ta.trend import EMAIndicator
import pandas_ta as pdta


def Data_Get(Symbol,timeframe):
    try:
        data = {
                    "symbol":Symbol,
                    "resolution":timeframe,
                    "date_format":"1",
                    "range_from":(datetime.now() - timedelta(days=100)).strftime('%Y-%m-%d'),
                    "range_to":datetime.now().strftime('%Y-%m-%d'),
                    "cont_flag":"0"
                }
        row_data =  fyers.history(data=data)
        df = pd.DataFrame.from_dict(row_data['candles'])
        columns_name = ['Datetime','Open','High','Low','Close','Volume']
        df.columns = columns_name
        df['Datetime'] = pd.to_datetime(df['Datetime'], unit='s')
        df['Datetime'] = df['Datetime'].dt.tz_localize(pytz.utc).dt.tz_convert('Asia/Kolkata')
        df['Datetime'] = df['Datetime'].dt.tz_localize(None)


        df['Pivot'] = (df['High'] + df['Low'] + df['Close']) / 3
        df['Candle'] = df.apply(lambda row: 'Green' if row['Close'] >= row['Open'] else 'Red', axis=1)
        df['5EMA'] = EMAIndicator(close=df['Close'], window=5, fillna=False).ema_indicator()
        df['20EMA'] = EMAIndicator(close=df['Close'], window=20, fillna=False).ema_indicator()
        df['50EMA'] = EMAIndicator(close=df['Close'], window=50, fillna=False).ema_indicator()
        df['200EMA'] = EMAIndicator(close=df['Close'], window=200, fillna=False).ema_indicator()
    
        super_trend = pdta.supertrend(high=df['High'], low=df['Low'], close=df['Close'], length=50, multiplier=4)
        df['Super_Trend'] = super_trend['SUPERTd_50_4.0']
        
        df["Max"] = df['High'].rolling(window=40, min_periods=1).max()
        df["Min"] = df['Low'].shift(1).rolling(window=40, min_periods=1).min()
    
    
        # ----------------------------------FEBONACY RETECHMENT----------------------------------
        def find_closest_fib_level(close_price, fib_levels, max_val, min_val):
            levels = [min_val + level * (max_val - min_val) for level in fib_levels]
            distances = [abs(level - close_price) for level in levels]
            closest_index = distances.index(min(distances))
            closest_level = fib_levels[closest_index]
            distance = round(distances[closest_index], 2)
            return closest_level, distance
    
        fib_levels = [1.33, 1.0, 0.67, 0.61, 0.33, -0.33]
        
        df['Close_Feb_Ret'] = df.apply(lambda row: find_closest_fib_level(row['Close'], fib_levels, row['Max'], row['Min']), axis=1)
        
        #  ----------------------------------CHECK EMA TOUCH----------------------------------
        def Check_EMA_Test(row, n=1):
            if  pd.notna(row['50EMA']) and pd.notna(row['200EMA']):
                ema_50test = row['Low'] - n <= row['50EMA'] <= row['High'] + n
                ema_200test = row['Low'] - n <= row['200EMA'] <= row['High'] + n
                return ema_50test, ema_200test
            else:
                return False, False
        df[['50EMA_Test', '200EMA_Test']] = df.apply(Check_EMA_Test, axis=1, result_type='expand')
        df['50EMA_Test'] = df['50EMA_Test'].rolling(window=2).apply(lambda x: any(x), raw=True).astype(bool)
        df['200EMA_Test'] = df['200EMA_Test'].rolling(window=2).apply(lambda x: any(x), raw=True).astype(bool)
    
        # ----------------------------CHEKING ENTRYES-------------------------
        
        # ----------------------------------5 EMA CE PE---------------------------------- 
        ce_5ema = (df['High'].shift(1) < df['5EMA'].shift(1)) & (df['Close'] >= df['5EMA'])
        pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) & (df['Close'] <= df['5EMA'])
        
        df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='0')
        # ----------------------------------15 EMA CE PE---------------------------------- 
        ce_ema_crossover = (df['5EMA'] > df['20EMA']) & (df['5EMA'].shift(1) <= df['20EMA'].shift(1))
        pe_ema_crossover = (df['5EMA'] < df['20EMA']) & (df['5EMA'].shift(1) >= df['20EMA'].shift(1))
        
        df['Signal_EMA_CROSSOVER'] = np.select([ce_ema_crossover, pe_ema_crossover],['CE', 'PE'], default='0')
    
        # ----------------------------------FINAL ENTRY STATUS---------------------------------- 
    
        df['Feb&Trend'] = df.apply(lambda row: 'CE' if row['Super_Trend'] == 1 and row['Close_Feb_Ret'][0] <= 0.7 and row['Close_Feb_Ret'][1] <= 40 else 'PE', axis=1)
    
        ce_5ema = ((df['Feb&Trend'] == "CE") & (df['Signal_5EMA'] == "CE")) | ((df['Feb&Trend'] == "CE") & (df['Signal_EMA_CROSSOVER'] == "CE")) | ((df['50EMA_Test'] | df['200EMA_Test']) & (df['Feb&Trend'] == "CE"))
        pe_5ema = ((df['Feb&Trend'] == "PE") & (df['Signal_5EMA'] == "PE")) | ((df['Feb&Trend'] == "PE") & (df['Signal_EMA_CROSSOVER'] == "PE")) | ((df['50EMA_Test'] | df['200EMA_Test']) & (df['Feb&Trend'] == "PE"))
    
        df['5EMA_Feb&Trend_SIGNAL'] = np.select([ce_5ema, pe_5ema], ['CE', 'PE'], default='NONE')
    
        
        delete_column_name = ['50EMA_Test','200EMA_Test','Max','Min','Feb&Trend','Signal_EMA_CROSSOVER','Signal_5EMA','200EMA','50EMA']
        df = df.drop(columns= delete_column_name)
        df = df.dropna()
    except :
        df = pd.DataFrame([])
    return df
    
# Symbol = "NSE:NIFTY50-INDEX"
# df = Data_Get(Symbol,15)

# live

In [248]:
def Live_data(data):
    ltp = data['ltp']
    
    t = time.localtime()
    csec = time.strftime("%S", t)
    print("[ONLINE]")
    
    if int(csec) in [11,41]:
        print("Just Update")
        df = Data_Get(data['symbol'],15)
        status = df.iloc[-1]['5EMA_Feb&Trend_SIGNAL'],df.iloc[-1]['Datetime']
        print(status)
        if status[0] == "CE":
            print("[CE ENTRY]")
        if status[0] == "PE":
            print("[PE ENTRY]")      
    else:
        print("[OFFLINE]")
        return False

  

In [249]:
from fyers_apiv3.FyersWebsocket import data_ws

def onmessage(message):
    Live_data(message)
    # print("Response:", message['ltp'])

def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"
    symbols = ['NSE:NIFTY50-INDEX']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    fyers.keep_running()


def Subscribe():
    fyers = data_ws.FyersDataSocket(
    access_token=access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=False,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=False,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
    )
    
    fyers.connect()


from datetime import datetime, time

now = datetime.now().time()
start_time = time(9, 20)
end_time = time(23, 30)

if start_time <= now <= end_time:
    Subscribe()
else:
    print("[MARKET CLOSE]")

Error: KeyError: The key 'ltp' is missing in the response.
Error: KeyError: The key 'ltp' is missing in the response.
Error: KeyError: The key 'ltp' is missing in the response.
Error: type object 'datetime.time' has no attribute 'localtime'
Error: Connection to remote host was lost.
Connection closed: {'code': 200, 'message': 'Connection Closed', 's': 'ok'}
Error: Connection to remote host was lost.
Connection closed: {'code': 200, 'message': 'Connection Closed', 's': 'ok'}
Error: Connection to remote host was lost.
Connection closed: {'code': 200, 'message': 'Connection Closed', 's': 'ok'}
Error: Connection to remote host was lost.
Connection closed: {'code': 200, 'message': 'Connection Closed', 's': 'ok'}


In [247]:
def Unsubscribe():
    data_type = "SymbolUpdate"
    symbols_to_unsubscribe = ['NSE:NIFTY50-INDEX']
    fyers.unsubscribe(symbols=symbols_to_unsubscribe, data_type=data_type)